In [177]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import time, random
from requests.exceptions import RequestException
from tqdm import tqdm  # Optional: For progress bars
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
from typing import Dict, List, Optional, Tuple
from requests.exceptions import RequestException, HTTPError
import pandas as pd
from tqdm import tqdm
import random


In [2]:
def fetch_url(url, headers, max_retries=3, timeout=10, sleep_between_retries=2):
    """
    Fetches the content of a URL with retries.

    Args:
        url (str): The URL to fetch.
        headers (dict): HTTP headers to include in the request.
        max_retries (int): Maximum number of retry attempts.
        timeout (int): Timeout for the HTTP request.
        sleep_between_retries (int): Seconds to wait between retries.

    Returns:
        requests.Response or None: The HTTP response if successful, else None.
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except RequestException as e:
            print(f"Attempt {attempt + 1} failed for URL: {url}. Error: {e}")
            if attempt < max_retries - 1:
                time.sleep(sleep_between_retries)
            else:
                print(f"Failed to fetch data for {url} after {max_retries} attempts.")
                return None


In [3]:
def parse_html(content):
    """
    Parses HTML content using BeautifulSoup.

    Args:
        content (str): HTML content as a string.

    Returns:
        BeautifulSoup: Parsed HTML.
    """
    return BeautifulSoup(content, 'html.parser')


In [244]:
def get_eps_forecast(company, current_year = 2025):
    """
    Retrieves EPS forecasts for a given company from StockAnalysis.

    Args:
        company (str): The stock ticker symbol.

    Returns:
        dict or None: Dictionary containing EPS for 2024 and 2025, or None if failed.
    """
    url = f"https://stockanalysis.com/stocks/{company}/forecast/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                      ' Chrome/91.0.4472.124 Safari/537.36'
    }

    response = fetch_url(url, headers)
    if not response:
        return None

    soup = parse_html(response.text)
    table = soup.find('table', {'class': 'w-full whitespace-nowrap border border-gray-200 text-right text-sm dark:border-dark-700 sm:text-base'})

    if not table:
        print(f"No forecast table found for {company}")
        return None

    headers_table = [header.get('title', header.text).strip() for header in table.find_all('th')]
    rows = table.find_all('tr')[1:]  # Skip header row
    data = []

    for row in rows:
        cells = row.find_all('td')
        cell_data = [cell.text.strip() for cell in cells]
        # Ensure data aligns with headers
        cell_data += [''] * (len(headers_table) - len(cell_data))
        data.append(cell_data[:len(headers_table)])

    if not data:
        print(f"No forecast data extracted for {company}")
        return None
  
    df = pd.DataFrame(data, columns=headers_table)
    # print(df)

    try:
        eps_row = df[df['Fiscal Year'] == 'EPS']
        eps_current_year = float(eps_row[str(current_year)].values[0]) if not eps_row.empty and str(current_year) in df.columns else 0
        eps_next_year = float(eps_row[str(current_year+1)].values[0]) if not eps_row.empty and str(current_year+1) in df.columns else 0
        eps_past_year = float(eps_row[str(current_year-1)].values[0]) if not eps_row.empty and str(current_year+1) in df.columns else 0
        eps_past_year_5 = float(eps_row[str(current_year-5)].values[0]) if not eps_row.empty and str(current_year+1) in df.columns else 0
        print('eps_past_year:', eps_past_year)
        print('eps_past_year_5:', eps_past_year_5)
        if eps_past_year_5 > 0 and eps_past_year > 0:
            eps_growth = round(((eps_past_year / eps_past_year_5) ** (1 / 5) - 1) * 100, 2)
        else:
            eps_growth = None
        # print('eps growth:', eps_growth)
    except (KeyError, ValueError, IndexError) as e:
        print(f"Error extracting EPS data for {company}: {e}")
        eps_current_year, eps_next_year = None, None
    current_year_suffix = str(current_year)[-2:]
    next_year_suffix = str(current_year + 1)[-2:]
    return {
        f"eps_{current_year_suffix}" : eps_current_year,
        f"eps_{next_year_suffix}" : eps_next_year,
        "past_eps_growth" : eps_growth
    }


In [235]:
get_eps_forecast('ALK')  # Example usage

eps_past_year: 3.08
eps_past_year_5: -10.72


{'eps_25': 5.77, 'eps_26': 7.44, 'past_eps_growth': None}

In [241]:
import requests
from bs4 import BeautifulSoup
import re

def extract_percentage(text):
    """
    Extracts percentage value from text and converts to float.
    
    Args:
        text (str): Text containing percentage
        
    Returns:
        float: Percentage value as float, or None if not found
    """
    if not text:
        return None
    match = re.search(r'([-+]?\d*\.?\d+)%', text)
    if match:
        return float(match.group(1))
    return None

def get_growth_forecasts(company):
    """
    Retrieves 5-year growth forecasts from StockAnalysis.com.
    
    Args:
        company (str): The stock ticker symbol
        
    Returns:
        dict or None: Dictionary containing revenue and EPS growth forecasts
    """
    url = f"https://stockanalysis.com/stocks/{company}/statistics/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
    }
    
    try:
        response = fetch_url(url, headers)
        if not response:
            return None
            
        soup = parse_html(response.text)
        
        # Find growth forecast values
        forecasts = {
            'revenue_growth_5y': None,
            'eps_growth_5y': None
        }
        
        # Find elements containing the forecasts
        for row in soup.find_all('tr'):
            cells = row.find_all('td')
            if not cells:
                continue
                
            label_cell = cells[0].get_text(strip=True)
            if 'Revenue Growth Forecast' in label_cell and '5Y' in label_cell:
                forecasts['revenue_growth_5y'] = extract_percentage(cells[-1].get_text(strip=True))
            elif 'EPS Growth Forecast' in label_cell and '5Y' in label_cell:
                forecasts['eps_growth_5y'] = extract_percentage(cells[-1].get_text(strip=True))
        
        # Verify we found both metrics
        if forecasts['revenue_growth_5y'] is None and forecasts['eps_growth_5y'] is None:
            print(f"No growth forecasts found for {company}")
            forecasts['revenue_growth_5y'] = 0
            forecasts['eps_growth_5y'] = 0
            
        return forecasts
        
    except Exception as e:
        print(f"Error extracting growth forecasts: {e}")
        return None

# # Example usage
# def main():
#     ticker = "AMD"
#     forecasts = get_growth_forecasts(ticker)
    
#     if forecasts:
#         print(f"\nGrowth Forecasts for {ticker}:")
#         print(f"Revenue Growth (5Y): {forecasts['revenue_growth_5y']}%")
#         print(f"EPS Growth (5Y): {forecasts['eps_growth_5y']}%")
#         return forecasts
#     return None

# if __name__ == "__main__":
#     main()

In [242]:
get_growth_forecasts('ALK')  # Example usage

No growth forecasts found for ALK


{'revenue_growth_5y': 0, 'eps_growth_5y': 0}

In [22]:
import yfinance as yf

# Define the stock ticker
ticker = yf.Ticker("AAPL")
# Accessing various financial ratios
pe_ratio = ticker.info['forwardPE']  # Forward P/E ratio
pb_ratio = ticker.info['priceToBook']  # Price-to-Book ratio
de_ratio = ticker.info['debtToEquity']  # Debt-to-Equity ratio
roe = ticker.info['returnOnEquity']  # Return on Equity

# Print the results
print(f"Forward P/E Ratio: {pe_ratio}")
print(f"Price-to-Book Ratio: {pb_ratio}")
print(f"Debt-to-Equity Ratio: {de_ratio}")
print(f"Return on Equity: {roe}")


Forward P/E Ratio: 28.715042
Price-to-Book Ratio: 63.07141
Debt-to-Equity Ratio: 209.059
Return on Equity: 1.5741299


In [190]:
import yfinance as yf
def get_market_cap_and_price(ticker):
    """
    Retrieves the market capitalization and current price for a given stock ticker.

    Args:
        ticker (str): The stock ticker symbol.

    Returns:
        tuple: A tuple containing the market capitalization and current price, or None if failed.
    """
    try:
        stock = yf.Ticker(ticker)
        
        market_cap = round(int(stock.info['marketCap'])/1000000000, 2) if 'marketCap' in stock.info else "NA"
        current_price = stock.info['currentPrice'] if 'currentPrice' in stock.info else "NA"
        beta_value = stock.info['beta'] if 'beta' in stock.info else "NA"
    
        return {
            'ticker': ticker,
            '市值': str(market_cap)+"B",
            '股價': current_price,
            'Beta': (beta_value)
        }
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")
        return None


In [108]:
get_market_cap_and_price('NU')  # Example usage

{'市值': '64.6B', '股價': 13.44, 'Beta': 'NA'}

In [218]:
ticker = 'BA'
eps_data = get_eps_forecast(ticker)
growth_data = get_growth_forecasts(ticker)
stock_data = get_market_cap_and_price(ticker)
stock_data.update(eps_data)
stock_data.update(growth_data)
stock_data

{'ticker': 'BA',
 '市值': '134.51B',
 '股價': 179.53,
 'Beta': 1.572,
 'eps_25': 0.22,
 'eps_26': 4.78,
 'past_eps_growth': -4.45,
 'revenue_growth_5y': 10.75,
 'eps_growth_5y': None}

In [169]:
stock_data

{'市值': '3572.85B',
 '股價': 237.59,
 'Beta': 1.24,
 'eps_25': 7.51,
 'eps_26': 8.41,
 'past_eps_growth': 13.14,
 'revenue_growth_5y': 7.85,
 'eps_growth_5y': 18.45}

In [178]:
TICKER_TO_COMPANY: Dict[str, str] = {
    # 大科技
    "goog": "alphabet",
    "meta": "meta-platforms",
    "amzn": "amazon",
    "nflx": "netflix",
    "aapl": "apple",
    "msft": "microsoft",
    "tsla": "tesla",
    "adbe": "adobe",

    # 航空郵輪
    "aal": "american-airlines",
    "luv": "southwest-airlines",
    "dal": "delta-air-lines",
    "ual": "united-airlines",
    "alk": "alaska-air-group",
    "ba": "boeing",
    "eadsy": "airbus",
    "ccl": "carnival",
    "rcl": "royal-caribbean",

    # 銀行
    "bac": "bank-of-america",
    "jpm": "jpmorgan-chase",
    "gs": "goldman-sachs",
    "c": "citigroup",
    "wfc": "wells-fargo",
    "blk": "blackrock",
    "nu": "nu-holdings",
    "sofi": "sofi",

    # 傳統
    "dis": "walt-disney",
    "isrg": "intuitive-surgical",
    "unh": "unitedhealth-group",
    "abbv": "abbvie",
    "cvs": "cvs-health",
    "trv": "the-travelers-companies",

    # 支付
    "ma": "mastercard",
    "v": "visa",
    "pypl": "paypal",
    "axp": "american-express",

    # 零售
    "lulu": "lululemon-athletica",
    "gps": "gap",
    "cost": "costco-wholesale",
    "pg": "procter-&-gamble",
    "kr": "kroger",
    "jwn": "nordstrom",
    "nke": "nike",
    "dg": "dollar-general",
    "fl": "foot-locker",
    "lvmhf": "lvmh",
    "el": "estée-lauder",
    "pvh": "pvh-corp.",
    "tpr": "tapestry",

    # 食品
    "tsn": "tyson-foods",
    "mcd": "mcdonald's",
    "sbux": "starbucks",
    "ko": "coca-cola",
    "pep": "pepsi-co",
    "cmg": "chipotle-mexican-grill",
    "yum": "yum!-brands",
    "dpz": "domino's-pizza",
    "cake": "the-cheesecake-factory",
    "jack": "jack-in-the-box",
    "play": "dave-&-busters",
    "fizz": "national-beverage",
    "blmn": "bloomin'-brands",
    "denn": "denny's",
    "din": "dine-brands-global",

    # 半導體
    "nvda": "nvidia",
    "tsm": "taiwan-semiconductor-(tsmc)",
    "amd": "advanced-micro-devices",
    "qcom": "qualcomm",
    "mu": "micron-technology",
    "intc": "intel",
    "asml": "asml-holding",
    "swks": "skyworks-solutions",
    "qrvo": "qorvo",
    "avgo": "broadcom",
    "amat": "applied-materials",
    "mrvl": "marvell-technology",
    "arm": "arm-holdings",
    "cls": "celestica",
    "dell": "dell-technologies",
    "hpe": "hewlett-packard-enterprise",

    # 原油
    "cvx": "chevron",
    "vlo": "valero-energy",
    "cop": "conocophillips",
    "xom": "exxon-mobil",
    "oxy": "occidental-petroleum",
    "cpe": "callon-petroleum",
    "enb": "enbridge",

    # 旅遊
    "bkng": "booking-holdings",
    "expe": "expedia-group",
    "mar": "marriott-international",
    "hlt": "hilton-worldwide",
    "abnb": "airbnb",
    "h": "hyatt-hotels",
    "wynn": "wynn-resorts",
    "ihg": "intercontinental-hotels",
    "lvs": "las-vegas-sands",
    "mgm": "mgm-resorts",

    # 工業
    "nee": "nextera-energy",
    "de": "deere-&-company",
    "hd": "home-depot",
    "apd": "air-products-&-chemicals",
    "cat": "caterpillar",
    "etn": "eaton-corporation",
    "hon": "honeywell",
    "wm": "waste-management",
    "ge": "general-electric",
    "mmm": "3m-company",
    "sum": "summit-materials",
    "x": "u.s.-steel",
    "enph": "enphase-energy",
    "sedg": "solaredge",
    "fslr": "first-solar",

    # SaaS
    "sap": "sap-se",           # Ticker is sometimes just 'sap'
    "cflt": "confluent",
    "acn": "accenture",
    "bsx": "boston-scientific",
    "shop": "shopify",
    "crm": "salesforce",
    "ddog": "datadog",
    "now": "servicenow",
    "intu": "intuit",
    "sq": "block-(square)",
    "wdays": "workday",
    "snow": "snowflake",
    "mdb": "mongodb",
    "okta": "okta",
    "adsk": "autodesk",
    "ttd": "the-trade-desk",
}


In [181]:
def parse_pe_ratios(ticker: str) -> List[float]:
    """
    Parse PE ratios from the given HTML content.

    Args:
        html_content (str): HTML content of the webpage.

    Returns:
        List[float]: List of PE ratios extracted.
    """
    company = TICKER_TO_COMPANY.get(ticker.lower())
    url = f"https://www.macrotrends.net/stocks/charts/{ticker.upper()}/{company}/pe-ratio"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
    }
    
    try:
        response = fetch_url(url, headers)
        if not response:
            return None
            
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find("table", class_="table")

        if not table:
            print("PE ratio table not found in the HTML content.")
            return []

        rows = table.find("tbody").find_all("tr")
        pe_ratios = []

        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 4:
                pe_ratio_str = cells[3].get_text(strip=True)
                # Handle cases like 'N/A' or empty strings
                try:
                    pe_ratio = float(pe_ratio_str.replace(",", ""))
                    pe_ratios.append(pe_ratio)
                except ValueError:
                    # Skip invalid PE ratio values
                    continue

        return pe_ratios
    except Exception as e:
        print(f"Error extracting growth forecasts: {e}")
        return None

In [183]:
# -------------------------------
# Statistical Computations
# -------------------------------

def compute_iqr_statistics(pe_array: np.ndarray) -> Tuple[float, float, float]:
    """
    Compute Q1, Q3, and IQR for the given PE ratios.

    Args:
        pe_array (np.ndarray): Array of PE ratios.

    Returns:
        Tuple[float, float, float]: Q1, Q3, and IQR values.
    """
    q1 = np.percentile(pe_array, 25)
    q3 = np.percentile(pe_array, 75)
    iqr = q3 - q1
    return q1, q3, iqr

def filter_outliers(pe_array: np.ndarray, q1: float, q3: float, iqr: float) -> np.ndarray:
    """
    Filter out outliers based on IQR.

    Args:
        pe_array (np.ndarray): Array of PE ratios.
        q1 (float): First quartile.
        q3 (float): Third quartile.
        iqr (float): Interquartile range.

    Returns:
        np.ndarray: Filtered array without outliers.
    """
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return pe_array[(pe_array >= lower_bound) & (pe_array <= upper_bound)]

In [186]:
def analyze_pe_ratios(pe_ratios: List[float], num_latest: int = 20) -> Tuple[Optional[float], Optional[float]]:
    """
    Analyze PE ratios by computing mean before and after outlier removal.

    Args:
        pe_ratios (List[float]): List of PE ratios.
        num_latest (int): Number of latest PE ratios to consider.

    Returns:
        Tuple[Optional[float], Optional[float]]: Mean PE after outlier removal and original mean.
    """
    if not pe_ratios:
        return None, None

    # Consider the latest `num_latest` PE ratios
    latest_pe_ratios = pe_ratios[:num_latest]
    pe_array = np.array(latest_pe_ratios)

    # Compute IQR statistics
    q1, q3, iqr = compute_iqr_statistics(pe_array)
    filtered_pe = filter_outliers(pe_array, q1, q3, iqr)

    # Compute means
    if filtered_pe.size > 0:
        median_filtered_pe = np.median(filtered_pe)
        return median_filtered_pe
    else:
        return np.median(pe_array)

In [187]:
analyze_pe_ratios(parse_pe_ratios('AAPL'))

27.775

In [238]:
def calculate_valuations(stock_data, current_year):
    """
    Calculates various valuations and differences based on stock data.

    Args:
        stock_data (dict): Contains 'eps_2025', 'eps_2026', '市值', '股價'.
        growth_data (dict): Contains '前五年' and '後五年' growth rates.
        pe_median (float): The five-year median PE ratio.
        current_year (int): The current year (e.g., 2025).

    Returns:
        dict: Calculated valuations and differences.
    """
    valuations = {}
    # Derive suffixes
    current_year_suffix = str(current_year)[-2:]
    next_year_suffix = str(current_year + 1)[-2:]

    growth = stock_data.get('eps_growth_5y', 0)
    if growth == None:
        growth = 0
    print("growth: ",growth)
    valuations['預估PE'] = growth 
    if 0 < growth < 5:
        valuations['預估PE'] *= 0.8
    elif 5 <= growth < 10:
        valuations['預估PE'] *= 1.0
    elif 10 <= growth < 15:
        valuations['預估PE'] *= 1.1
    elif 15 <= growth < 20:
        valuations['預估PE'] *= 1.2
    elif 20 <= growth < 30:
        valuations['預估PE'] *= 1.5
    else:
        valuations['預估PE'] *= 2.0
    valuations['預估PE'] = round(valuations['預估PE'], 2)
    print('after growth: ', valuations['預估PE'])
    # Calculate EPS and Reasonable Price for current and next year
    valuations[f"{current_year_suffix}年EPS"] = stock_data.get(f'eps_{current_year_suffix}', 0)
    valuations[f"{current_year_suffix}年合理價"] = round(valuations[f"{current_year_suffix}年EPS"] * valuations['預估PE'])

    valuations[f"{next_year_suffix}年EPS"] = stock_data.get(f'eps_{next_year_suffix}', 0)
    valuations[f"{next_year_suffix}年合理價"] = round(valuations[f"{next_year_suffix}年EPS"] * valuations['預估PE'])

    # Five-year PE Median and Median Valuation
    valuations["五年PE MEDIAN"] = analyze_pe_ratios(parse_pe_ratios(stock_data['ticker']))
    valuations["五年PE中位價"] = round(valuations["五年PE MEDIAN"] * valuations[f"{current_year_suffix}年EPS"])

    # Valuation Comparison for Current Year
    if stock_data.get('股價', 0) > valuations["五年PE中位價"]:
        valuations[f"{current_year_suffix}年估值"] = "高估"
    else:
        valuations[f"{current_year_suffix}年估值"] = "低估"

    if valuations["五年PE中位價"] != 0:
        valuations[f"{current_year_suffix}年相差百分比"] = f"{round(100 * (valuations['五年PE中位價'] - stock_data.get('股價', 0)) / valuations['五年PE中位價'])}%"
    else:
        valuations[f"{current_year_suffix}年相差百分比"] = "N/A"

    # Five-year PE Median Valuation for Next Year
    valuations[f"{next_year_suffix}年PE中位價"] = round(valuations["五年PE MEDIAN"] * valuations[f"{next_year_suffix}年EPS"])

    # Valuation Comparison for Next Year
    if stock_data.get('股價', 0) > valuations[f"{next_year_suffix}年PE中位價"]:
        valuations[f"{next_year_suffix}年估值"] = "高估"
    else:
        valuations[f"{next_year_suffix}年估值"] = "低估"

    if valuations[f"{next_year_suffix}年PE中位價"] != 0:
        valuations[f"{next_year_suffix}年相差百分比"] = f"{round(100 * (valuations[f'{next_year_suffix}年PE中位價'] - stock_data.get('股價', 0)) / valuations[f'{next_year_suffix}年PE中位價'])}%"
    else:
        valuations[f"{next_year_suffix}年相差百分比"] = "N/A"

    return valuations


In [207]:
valuations = calculate_valuations(stock_data, 2025)
valuations

{'預估PE': 33.51,
 '25年EPS': 20.85,
 '25年合理價': 699,
 '26年EPS': 23.59,
 '26年合理價': 791,
 '五年PE MEDIAN': 43.93,
 '五年PE中位價': 916,
 '25年估值': '低估',
 '25年相差百分比': '51%',
 '26年PE中位價': 1036,
 '26年估值': '低估',
 '26年相差百分比': '57%'}

In [78]:
current_year = 2025
current_year_suffix = str(current_year)[-2:]
next_year_suffix = str(current_year + 1)[-2:]

In [213]:
def process_company(company, current_year=2025):
    """
    Processes data for a single company.

    Args:
        company (str): The stock ticker symbol.
        current_year_suffix (str): Last two digits of the current year.
        next_year_suffix (str): Last two digits of the next year.

    Returns:
        dict or None: Dictionary containing all required data, or None if failed.
    """
    current_year_suffix = str(current_year)[-2:]
    next_year_suffix = str(current_year + 1)[-2:]
    # Fetch data from various sources
    eps_forecast = get_eps_forecast(company)
    growth_data = get_growth_forecasts(company)
    if not eps_forecast:
        return None
    pe_median = analyze_pe_ratios(parse_pe_ratios(company))
    print(pe_median)
    stock_data = get_market_cap_and_price(company)
    if not stock_data:
        return None
    stock_data.update(eps_forecast)
    stock_data.update(growth_data)
    # Perform calculations
    valuations = calculate_valuations(
        stock_data=stock_data,
        current_year= current_year
    )
    print(valuations)
    # Compile all data into a single dictionary
    company_data = {
        "Revenue Growth Forecast (5Y)": str(stock_data.get("revenue_growth_5y", 0))+"%",
        "EPS Growth Forecast (5Y)": str(stock_data.get("eps_growth_5y", 0))+"%",
        "EPS Growth Past 5 Years": str(stock_data.get("past_eps_growth", 0))+"%",
        "預估PE": valuations.get("預估PE", 0),
        f"{current_year_suffix}年EPS": valuations.get(f"{current_year_suffix}年EPS", 0),
        f"{current_year_suffix}年合理價": valuations.get(f"{current_year_suffix}年合理價", 0),
        f"{next_year_suffix}年EPS": valuations.get(f"{next_year_suffix}年EPS", 0),
        f"{next_year_suffix}年合理價": valuations.get(f"{next_year_suffix}年合理價", 0),
        "五年PEMEDIAN": valuations.get("五年PE MEDIAN", 0),
        "五年PE中位價": valuations.get("五年PE中位價", 0),
        "市值": stock_data.get("市值", "N/A"),
        "股價": stock_data.get("股價", 0),
        f"{current_year_suffix}年估值": valuations.get(f"{current_year_suffix}年估值", "N/A"),
        f"{current_year_suffix}年相差百分比": valuations.get(f"{current_year_suffix}年相差百分比", "N/A"),
        f"{next_year_suffix}年PE中位價": valuations.get(f"{next_year_suffix}年PE中位價", 0),
        f"{next_year_suffix}年估值": valuations.get(f"{next_year_suffix}年估值", "N/A"),
        f"{next_year_suffix}年相差百分比": valuations.get(f"{next_year_suffix}年相差百分比", "N/A"),
    }

    return company_data


In [231]:
process_company('ALK', 2025)  # Example usage

TypeError: type complex doesn't define __round__ method

In [96]:
def process_industry(industry, companies, current_year, col, ind):
    """
    Processes all companies within a specific industry.

    Args:
        industry (str): The industry name.
        companies (set): A set of company ticker symbols.
        current_year_suffix (str): Last two digits of the current year.
        next_year_suffix (str): Last two digits of the next year.
        col (list): Column names.
        ind (list): Indices for the DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing processed data for all companies in the industry.
    """
    # Initialize DataFrame with predefined columns
    ser = pd.Series(col, index=ind)
    industry_df = pd.DataFrame(columns=["股票代號"] + [company for company in companies])
    industry_df["股票代號"] = ser

    for company in companies:
        print(f"Processing company: {company}")
        company_data = process_company(company, current_year_suffix, next_year_suffix)
        if not company_data:
            print(f"Skipping company {company} due to incomplete data.")
            continue

        # Convert company_data dict to DataFrame
        stock_df = pd.DataFrame(company_data, index=[0]).T
        stock_df = stock_df.rename(columns={0: company})
        stock_df.index.name = "股票代號"

        # Merge with industry DataFrame
        industry_df = pd.concat([industry_df, stock_df], axis=1)

    return industry_df


In [245]:
stock_list = {
        # "大科技": {"GOOG", "META", "AMZN", "NFLX", "AAPL", "MSFT", "TSLA", "ADBE"},
        # "航空郵輪": {"AAL", "LUV", "DAL", "UAL", "ALK", "BA", "CCL", "RCL"},
        
        # "銀行": {"BAC", "JPM", "GS", "C", "WFC", 'NU', 'SOFI'},
        # "傳統": {"DIS", "ISRG", "UNH", "ABBV", "CVS", 'TRV'},
        # "支付": {"MA", "V", "PYPL", "AXP"},
        "零售": {"LULU", "COST", "PG", "KR", "JWN", "NKE", "DG", "FL", "EL", "PVH", "TPR"},
        "食品": {"TSN", "MCD", "SBUX", "KO", "PEP", "CMG", "YUM", 'DPZ', 'CAKE', 'JACK', 'PLAY', 'FIZZ', 'BLMN', 'DENN', 'DIN'},
        "半導體": {'NVDA', "TSM", "AMD", "QCOM", "MU", "INTC", "ASML", "SWKS", "QRVO", "AVGO", "AMAT", 'MRVL', 'ARM', 'CLS', 'DELL', 'HPE'},
        "原油": {"CVX", "VLO", "COP", "XOM", "OXY", "CPE", "ENB"},
        "旅遊": {"BKNG", "EXPE", "MAR", "HLT", "ABNB", "H", "WYNN", "IHG", "LVS", "MGM"},
        "工業": {'NEE', "DE", "HD", "APD", "CAT", "ETN", "HON", "WM", "GE", "MMM", "SUM", "X", 'ENPH', 'SEDG', "FSLR"},
        'SaaS': {'SAP', 'CFLT', 'ACN', 'BSX', 'SHOP', 'ADBE', 'CRM', 'DDOG', 'NOW', 'INTU', 'SQ', 'WDAY', 'SNOW', 'MDB', 'OKTA', 'ADSK', 'TTD'}
    }

# EADSY, BLK, "LVMHF", GPS

In [216]:
def aggregate_company_data(stock_list, current_year=2025):
    """
    將所有公司的數據聚合成一個字典，每個產業對應一個轉置後的DataFrame。

    Args:
        stock_list (dict): 以產業為鍵，股票代碼集合為值的字典。
        current_year (int): 當前年份。

    Returns:
        dict: 以產業為鍵，轉置後的DataFrame為值的字典。
    """
    industry_dataframes = {}

    for industry, companies in stock_list.items():
        print(f"\n正在處理產業：{industry}，包含 {len(companies)} 家公司。")
        all_companies_data = []
        for company in tqdm(companies, desc=f"處理 {industry}", unit="公司"):
            print(f"正在處理公司：{company}")
            company_data = process_company(company, current_year =current_year)
            if company_data:
                # 添加元數據
                company_data['Industry'] = industry
                company_data['Company'] = company
                all_companies_data.append(company_data)
            else:
                print(f"{company} 的數據處理失敗。")
            time.sleep(random.uniform(10, 30))  # 隨機等待時間
            
        if all_companies_data:
            # 創建DataFrame
            df = pd.DataFrame(all_companies_data)
            # 添加Industry和Company列
            df.insert(0, 'Industry', df.pop('Industry'))
            df.insert(1, 'Company', df.pop('Company'))
            # 轉置DataFrame
            df_transposed = df.set_index(['Industry', 'Company']).transpose()
            # 保存到字典
            industry_dataframes[industry] = df_transposed
        else:
            print(f"{industry} 沒有可用的公司數據。")
        

    return industry_dataframes


In [246]:
df = aggregate_company_data(stock_list, 2025)  # Example usage


正在處理產業：零售，包含 11 家公司。


處理 零售:   0%|          | 0/11 [00:00<?, ?公司/s]

正在處理公司：JWN
eps_past_year: 0.82
eps_past_year_5: 3.18
10.22
growth:  32.43
after growth:  64.86
{'預估PE': 64.86, '25年EPS': 1.98, '25年合理價': 128, '26年EPS': 2.04, '26年合理價': 132, '五年PE MEDIAN': 10.22, '五年PE中位價': 20, '25年估值': '高估', '25年相差百分比': '-21%', '26年PE中位價': 21, '26年估值': '高估', '26年相差百分比': '-15%'}


處理 零售:   9%|▉         | 1/11 [00:13<02:10, 13.01s/公司]

正在處理公司：PVH
eps_past_year: 10.76
eps_past_year_5: 5.6
8.055
growth:  13.04
after growth:  14.34
{'預估PE': 14.34, '25年EPS': 11.83, '25年合理價': 170, '26年EPS': 12.47, '26年合理價': 179, '五年PE MEDIAN': 8.055, '五年PE中位價': 95, '25年估值': '低估', '25年相差百分比': '2%', '26年PE中位價': 100, '26年估值': '低估', '26年相差百分比': '7%'}


...

處理 食品:  33%|███▎      | 5/15 [01:48<03:12, 19.22s/公司]

正在處理公司：DENN
eps_past_year: 0.54
eps_past_year_5: -0.08
No growth forecasts found for DENN
14.415
growth:  0
after growth:  0.0


處理 食品:  33%|███▎      | 5/15 [01:53<03:46, 22.67s/公司]

Error fetching URL: 429 Client Error: Too Many Requests for url: https://www.macrotrends.net/stocks/charts/DENN/denny's/pe-ratio


TypeError: can't multiply sequence by non-int of type 'float'

In [132]:
import os
def save_to_excel(industry_dataframes, path):
    """
    將各產業的DataFrame保存到Excel文件中的不同工作表，並轉置數據。

    Args:
        industry_dataframes (dict): 以產業為鍵，轉置後的DataFrame為值的字典。
        path (str): 要保存的Excel文件路徑。
    """
    with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
        for industry, df in industry_dataframes.items():
            df.to_excel(writer, sheet_name=industry, index=True)
    print(f"\n所有數據已保存到 {path}")
save_to_excel(df, 'stock_data.xlsx')  # Example usage


所有數據已保存到 stock_data.xlsx
